In [27]:
%matplotlib widget
from sklearn import datasets, manifold
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.spatial.distance import pdist, squareform
from scipy import stats

In [2]:
n_samples = 500
x, y = datasets.make_s_curve( n_samples=n_samples )

idx = y.argsort()
y.sort()
x = x[idx]

In [51]:
def fit_plot( x, y, n_neighbors, n_components ):
    isomap = manifold.Isomap( n_neighbors=n_neighbors, n_components=n_components )
    y_isomap = isomap.fit_transform( x )
    
    lle = manifold.LocallyLinearEmbedding( n_neighbors=10, n_components=2 )
    y_lle = lle.fit_transform( x )
    
    fig, (ax1, ax2) = plt.subplots( 1, 2 )
    
    x1_vals = y_isomap[:, 0]
    y1_vals = y_isomap[:, 1]
    ax1.scatter( x1_vals, y1_vals, c=y )
    
    x2_vals = y_lle[:, 0]
    y2_vals = y_lle[:, 1] 
    ax2.scatter( x2_vals, y2_vals, c=y )
    plt.plot()
    return (y_isomap, y_lle)
    
def residual_variance( dx, dy ):
    corr = stats.spearmanr( dx, dy, axis=1 ).correlation
    return corr

In [8]:
y_isomap, y_lle = fit_plot( x, y, n_neighbors=10, n_components=2 )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [52]:
# AVALIACAO
# matriz de distancias original
dx = squareform( pdist( x ) )
# distancias da reducao
dy_isomap = squareform( pdist ( y_isomap ) )
dy_lle = squareform( pdist ( y_lle ) )

residual_variance( dx, dy_isomap )

array([[ 1.        ,  0.99838095,  0.91752127, ..., -0.71136582,
        -0.70698964, -0.71887795],
       [ 0.99838095,  1.        ,  0.93565517, ..., -0.71975558,
        -0.71578165, -0.72654041],
       [ 0.91752127,  0.93565517,  1.        , ..., -0.76961674,
        -0.76886612, -0.77022222],
       ...,
       [-0.71136582, -0.71975558, -0.76961674, ...,  1.        ,
         0.99984726,  0.99931523],
       [-0.70698964, -0.71578165, -0.76886612, ...,  0.99984726,
         1.        ,  0.99872934],
       [-0.71887795, -0.72654041, -0.77022222, ...,  0.99931523,
         0.99872934,  1.        ]])